In [11]:
import os

import sys
from pathlib import Path
import json
# Add the parent directory to the sys.path
sys.path.append(str(Path('/home/stirunag/work/github/CAPITAL/daily_pipeline/lib/python_scripts/').resolve()))


from collections import defaultdict, OrderedDict, Counter
import onnxruntime as ort
# from entity_linker import EntityLinker
from entity_tagger import count_lines_in_gzip, load_ner_model, batch_annotate_sentences, format_output_annotations, SECTIONS_MAP, extract_annotation
from tqdm import tqdm
import gzip
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)


PROVIDER = "europepmc"

# Mapping from abbreviation to full form
ENTITY_TYPE_MAP_1 = {
    "EM": "exp_methods", #methods
    "DS": "disease",
    "GP": "gene_protein",
    "GO": "go_term",
    "CD": "chemical",
    "OG": "organism"
}


# Helper Functions
def map_entity_type(abbrev, ENTITY_TYPE_MAP):
    """Map abbreviation to full form."""
    return ENTITY_TYPE_MAP.get(abbrev, abbrev.lower())


# Main function for processing article and generating JSONs
def process_article_generate_jsons(article_data):
    missing_ids = ['10291433',
   '10330421',
   '10330426',
   '10440280',
   '10449263',
   '10449359',
   '10449729',
   '10450820',
   '10451015',
   '10451022',
   '10454977',
   '10460088',
   '10470160',
   '10491428',
   '10524293',
   '10524363',
   '10524625',
   '10524694',
   '10524698',
   '10524704',
   '10525037',
   '10525040',
   '10526653',
   '10526679',
   '10526694',
   '10526894',
   '10527009',
   '10527090',
   '10527205',
   '10527225',
   '10527239',
   '10527335',
   '10529678',
   '10534201',
   '10561377',
   '10561594',
   '10563766',
   '10588561',
   '10625673',
   '10633838',
   '10857745',
   '10964084',
   '10964087',
   '10964088',
   '11133230',
   '11249782',
   '11291147',
   '11291532',
   '11291533',
   '11291551',
   '11297017',
   '11299584',
   '11300006',
   '11302177',
   '11302463',
   '11303546',
   '11303627',
   '11303630',
   '11303633',
   '11303731',
   '11307538',
   '11309959',
   '11309961',
   '11309984',
   '11310698',
   '11311446',
   '11312964',
   '11316869',
   '11316967',
   '11318990',
   '11319009',
   '11319015',
   '11319022',
   '11322925',
   '11325299',
   '11328371',
   '11328608',
   '11331836',
   '11333931',
   '11333948',
   '11333955',
   '11334057',
   '11337674',
   '11338587',
   '11340714',
   '11342024',
   '11342626',
   '11344235',
   '11349501',
   '11349506',
   '11349507',
   '11349508',
   '11352890',
   '11376101']
    
    pmcid = article_data.get("article_ids", {}).get("pmcid")
    ft_id = article_data.get("article_ids", {}).get("archive") or article_data.get("article_ids", {}).get("manuscript")

    if not pmcid and not ft_id:
        return None, None  # Skip article if no pmcid or ft_id

    if pmcid in missing_ids or ft_id in missing_ids:
        print(pmcid, ft_id)
        all_annotations = []
        for section_key, sentences in article_data.get("sections", {}).items():
            if section_key == "REF":
                continue  # Skip processing this section
            section = SECTIONS_MAP.get(section_key, "Other")
    
            # Pass the parallel flag to batch_annotate_sentences
            batch_annotations = batch_annotate_sentences(sentences, section, ner_model=ner_quantized, extract_annotation_fn=extract_annotation)
            if not batch_annotations:
                continue
    
            all_annotations.extend(batch_annotations)
    
        # Generate tags for annotations- this includes grounded terms and grounded codes.
        # Format matched and unmatched JSON structures
        # match_json, non_match_json = format_output_annotations(all_linked_annotations, pmcid=pmcid, ft_id=ft_id, PROVIDER=PROVIDER)
    
        # Return None if both JSONs are empty or have empty 'anns' lists
   
        print(all_annotations)
        # break



In [12]:
session_options = ort.SessionOptions()
session_options.intra_op_num_threads = 1  # Limit to a single thread
session_options.inter_op_num_threads = 1  # Limit to a single thread

# Directly assign the paths
input_file = "/home/stirunag/work/github/CAPITAL/daily_pipeline/notebooks/data/patch_2024_10_28_0.json.gz"  # Replace with your actual input file
model_path_quantised = "/home/stirunag/work/github/CAPITAL/model/europepmc"  # Replace with your actual model directory path

# Load PROVIDER_1 (europepmc) NER model
print("Loading NER model and tokenizer from " + model_path_quantised)
ner_quantized = load_ner_model(model_path_quantised, session_options)



total_lines = count_lines_in_gzip(input_file)

with gzip.open(input_file, "rt") as infile:
    for line in tqdm(infile, desc="Processing lines", unit="line", total=total_lines):
        article_data = json.loads(line)
        process_article_generate_jsons(article_data)


Loading NER model and tokenizer from /home/stirunag/work/github/CAPITAL/model/europepmc
Loading NER model from /home/stirunag/work/github/CAPITAL/model/europepmc
NER model from /home/stirunag/work/github/CAPITAL/model/europepmc loaded successfully.


Processing lines:   0%|          | 0/909 [00:00<?, ?line/s]

10470160 None


Processing lines:  22%|██▏       | 197/909 [00:04<00:10, 64.83line/s]

[{'type': 'DS', 'position': '4.17', 'prefix': 'indices to identify', 'exact': 'SDB', 'section': 'Abstract', 'postfix': 'in simulated responses.'}, {'type': 'DS', 'position': '6.45', 'prefix': 'AUCs to detect', 'exact': 'SDB', 'section': 'Abstract', 'postfix': ''}, {'type': 'DS', 'position': '9.27', 'prefix': 'indices to detect', 'exact': 'SDB', 'section': 'Methods', 'postfix': ''}, {'type': 'DS', 'position': '11.17', 'prefix': 'model to simulate', 'exact': 'SDB', 'section': 'Methods', 'postfix': 'in responses, as'}, {'type': 'DS', 'position': '13.23', 'prefix': 'affected by the', 'exact': 'SDB', 'section': 'Methods', 'postfix': 'factor, and one'}, {'type': 'DS', 'position': '13.38', 'prefix': '', 'exact': 'SDB', 'section': 'Methods', 'postfix': ''}, {'type': 'DS', 'position': '14.23', 'prefix': 'researcher, while the', 'exact': 'SDB', 'section': 'Methods', 'postfix': 'factor is the'}, {'type': 'DS', 'position': '14.34', 'prefix': '', 'exact': 'SDB', 'section': 'Methods', 'postfix': ''}

...

[]
11328371 None


Processing lines:  84%|████████▎ | 760/909 [00:36<00:04, 34.38line/s]

[]
11316869 None


Processing lines: 100%|██████████| 909/909 [00:37<00:00, 24.49line/s]

[]
